<a href="https://colab.research.google.com/github/Keerthana-hack24/ACM-JUMP-PROJECT-KEERTHANA/blob/main/MOVIE_PROJECT(new).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
pip install scikit-learn

In [51]:
import numpy as np
import pandas as pd
import difflib
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [52]:
pip install fuzzywuzzy

In [53]:
credits=pd.read_csv('/content/credits.csv').head(200)
titles=pd.read_csv('/content/titles.csv').head(200)


In [54]:
credits.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [55]:
titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [56]:
#selecting the important features for further analysis
selected_features = ['genres','description','release_year','imdb_score','age_certification','role','character']
print(selected_features)

['genres', 'description', 'release_year', 'imdb_score', 'age_certification', 'role', 'character']


In [57]:
for feature in selected_features:
    if feature in credits.columns:
        credits[feature] = credits[feature].fillna('')
    if feature in titles.columns:
        titles[feature] = titles[feature].fillna('')

In [58]:
final = pd.merge(
    titles,
    credits,
    left_on='id',
    right_on='id',
    how='left'  # Keeps all titles even if no credits exist
)

In [59]:
final.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,,NaN,0.600,NaN,NaN,NaN,NaN,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,3748.0,Robert De Niro,Travis Bickle,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,14658.0,Jodie Foster,Iris Steensma,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,7064.0,Albert Brooks,Tom,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,3739.0,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR


In [60]:
combined_features = (
    final['genres'].fillna('').astype(str) + ' ' +
    final['description'].fillna('').astype(str) + ' ' +
    final['release_year'].fillna('').astype(str) + ' ' +
    final['imdb_score'].fillna('').astype(str) + ' ' +
    final['age_certification'].fillna('').astype(str) + ' ' +
    final['role'].fillna('').astype(str)+' '+
    final['character'].fillna('').astype(str)
)
combined_features

,0
0,['documentation'] This collection includes 12 ...
1,"['drama', 'crime'] A mentally unstable Vietnam..."
2,"['drama', 'crime'] A mentally unstable Vietnam..."
3,"['drama', 'crime'] A mentally unstable Vietnam..."
4,"['drama', 'crime'] A mentally unstable Vietnam..."
...,...
388,"['documentation', 'history'] Many people first..."
389,['comedy'] Bill Hicks tells us how he feels ab...
390,"['comedy', 'documentation'] This is arguably T..."
391,['comedy'] Bill Hicks shows us his view on smo...


In [61]:
vectorizer = TfidfVectorizer()

In [62]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [63]:
print(feature_vectors)
similarity = cosine_similarity(feature_vectors)
print(similarity)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 18694 stored elements and shape (393, 3656)>
  Coords	Values
  (0, 993)	0.161919748761385
  (0, 3305)	0.14723123482198788
  (0, 715)	0.2587535993823358
  (0, 1694)	0.24205581897023887
  (0, 6)	0.11868619066700108
  (0, 3621)	0.11117786193819472
  (0, 3534)	0.09261633391026891
  (0, 1666)	0.11937833794526256
  (0, 1140)	0.22101910354405843
  (0, 2577)	0.2587535993823358
  (0, 1283)	0.2587535993823358
  (0, 2048)	0.13972290609318153
  (0, 2305)	0.05524790087024413
  (0, 3574)	0.1639290152340848
  (0, 284)	0.1048296604514114
  (0, 1463)	0.2587535993823358
  (0, 228)	0.0492372569590141
  (0, 2308)	0.2587535993823358
  (0, 977)	0.2587535993823358
  (0, 1687)	0.06091539605498746
  (0, 3283)	0.043881435868693856
  (0, 994)	0.2587535993823358
  (0, 1296)	0.24205581897023887
  (0, 566)	0.2587535993823358
  (0, 371)	0.142076792096977
  :	:
  (392, 664)	0.21583283300113157
  (392, 3246)	0.32374924950169737
  (392, 3194)	0.1079164165005

In [79]:
import difflib

def get_recommendations(final, similarity):

    choice = input("📺 Are you looking for a movie (m) or a show (s)? ").lower()
    title_input = input("🍿 Enter a movie/show you liked: ").strip()
    mood = input("🎭 Mood? (happy/sad/adventurous): ").lower()
    era = input("⏳ Preferred decade? (e.g., 1990s or 'any'): ").lower()
    rating = input("🔞 Max age rating? (G/PG/PG-13/R/NC-17 or 'any'): ").upper()
    actor = input("🌟 Favorite actor? (press Enter to skip): ").strip()
    surprise = input("🎲 Want a surprise? (y/n): ").lower()

    if choice == 'm':
        filtered_final = final[final['type'] == 'MOVIE']
    elif choice == 's':
        filtered_final = final[final['type'] == 'SHOW']
    else:
        print("❌ Invalid choice. Please enter 'm' or 's'.")
        return


    title_list = filtered_final['title'].tolist()
    matches = difflib.get_close_matches(title_input.lower(), [t.lower() for t in title_list], n=3, cutoff=0.5)

    if not matches:
        print("❌ No similar titles found.")
        return

    if len(matches) > 1:
        print("\nDid you mean:")
        for i, title in enumerate(matches, 1):
            print(f"{i}. {title.title()}")
        choice_num = input("Enter your choice number (1/2/3) or 0 to cancel: ")
        if choice_num == '0':
            return
        selected_title = matches[int(choice_num) - 1]
    else:
        selected_title = matches[0]


    try:
        idx_selected = filtered_final[filtered_final['title'].str.lower() == selected_title].index[0]
        similarity_scores = list(enumerate(similarity[idx_selected]))
    except:
        print("❌ Error finding recommendations.")
        return

    mood_keywords = {
        'happy': ['comedy', 'animation', 'musical'],
        'sad': ['drama', 'romance'],
        'adventurous': ['action', 'adventure', 'sci-fi']
    }
    rating_order = ['G', 'PG', 'PG-13', 'R', 'NC-17']

    filtered = []
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:100]  # Skip self

    seen_titles = set()

    for idx, score in similarity_scores:
        item = final.iloc[idx]


        if choice == 'm' and item['type'] != 'MOVIE':
            continue
        if choice == 's' and item['type'] != 'SHOW':
            continue

        include = True

        if mood in mood_keywords:
            genres = str(item.get('genres', '')).lower()
            if not any(word in genres for word in mood_keywords[mood]):
                include = False

        if era != 'any' and era.endswith('s') and era[:-1].isdigit():
            decade = int(era[:-1])
            year = item.get('release_year', 0)
            if not (decade <= year < decade + 10):
                include = False


        if rating.upper() != 'ANY' and rating in rating_order:
            allowed_ratings = rating_order[:rating_order.index(rating) + 1]
            if item.get('age_certification', '') not in allowed_ratings:
                include = False


        if actor and actor.lower() not in str(item.get('cast', '')).lower():
            include = False

        if include and item['title'] not in seen_titles:
            filtered.append((idx, score))
            seen_titles.add(item['title'])

        if len(filtered) >= 10:
            break

    if surprise == 'y':
        filtered = sorted(filtered, key=lambda x: x[1])


    print(f"\n🎬 Since you liked '{selected_title.title()}', you might enjoy:")
    for i, (idx, score) in enumerate(filtered, 1):
        item = final.iloc[idx]
        print(f"\n{i}. {item['title']} ({item.get('release_year', 'N/A')})")
        print(f"   ⭐ Similarity: {score:.2f} | {item.get('age_certification', 'Not Rated')} | {item.get('genres', 'Unknown')}")
        if actor:
            print(f"   🎭 Featuring: {', '.join(str(item.get('cast', '')).split()[:3])}...")
get_recommendations(final, similarity)

📺 Are you looking for a movie (m) or a show (s)? m
🍿 Enter a movie/show you liked: taxi
🎭 Mood? (happy/sad/adventurous): sad
⏳ Preferred decade? (e.g., 1990s or 'any'): any
🔞 Max age rating? (G/PG/PG-13/R/NC-17 or 'any'): any
🌟 Favorite actor? (press Enter to skip): 
🎲 Want a surprise? (y/n): 

Did you mean:
1. Fta
2. Titanic
3. Taxi Driver
Enter your choice number (1/2/3) or 0 to cancel: 2

🎬 Since you liked 'Titanic', you might enjoy:

1. Wild Tango (1993)
   ⭐ Similarity: 0.13 | R | ['drama', 'music', 'romance']

2. My Girl (1991)
   ⭐ Similarity: 0.12 | PG | ['drama', 'family', 'romance', 'comedy', 'european']

3. Manoranjan (1974)
   ⭐ Similarity: 0.12 |  | ['romance', 'drama', 'comedy']

4. Minsara Kanavu (1997)
   ⭐ Similarity: 0.11 | G | ['comedy', 'romance', 'drama']

5. Kuch Kuch Hota Hai (1998)
   ⭐ Similarity: 0.11 | G | ['romance', 'comedy', 'drama']

6. The Ryan White Story (1989)
   ⭐ Similarity: 0.10 |  | ['drama']

7. Damini (1993)
   ⭐ Similarity: 0.10 |  | ['drama', 